In [1]:
import numpy as np

In [2]:
file_path = "./ddi_with_type_latest.txt"
with open(file_path, "r") as f:
    readfile = f.read()

In [3]:
reflection_list = []
vector_set = set()
max_vector = 0
for item in readfile.split(sep="\n"):
    if len(item.split(sep="\t")) == 3:
        single_item = item.split(sep="\t")
        reflection_list.append(single_item)
        vector_set.add(single_item[1])
        vector_set.add(single_item[0])
        max_vector = max(max_vector, max(int(single_item[0]), int(single_item[1])))

## Q1 网络中节点和边的数量。

In [4]:
# number of nodes
len(vector_set)

1603

In [5]:
# number of edges
len(reflection_list)

38405

## Q2 找出度最大的前20个节点，并计算所有节点的平均度。

In [6]:
# Use dict to restore the degree of every nodes
node_degree = {str(i):0 for i in range(len(vector_set))}
for reflect in reflection_list:
    node_degree[reflect[0]] += 1
    node_degree[reflect[1]] += 1

In [7]:
# Order nodes by sum_degree and cut the top20
a = sorted(node_degree.items(), key=lambda x: x[1], reverse=True)
a[:20]

[('848', 194),
 ('350', 188),
 ('870', 186),
 ('118', 181),
 ('83', 177),
 ('972', 176),
 ('88', 174),
 ('1440', 173),
 ('1339', 166),
 ('446', 165),
 ('63', 163),
 ('140', 163),
 ('22', 162),
 ('187', 160),
 ('697', 159),
 ('441', 158),
 ('811', 157),
 ('33', 156),
 ('169', 155),
 ('266', 154)]

In [8]:
# Calculate the average degree of all nodes
average_degree = sum(node_degree.values())/len(vector_set)
average_degree

47.91640673736744

## Q3 统计网络中共有多少种不同类型的边，类型为47的边有多少条

In [9]:
# Count the nums of types
# Weight is equal to the index of type!!!
types = set()
cnt_47 = 0
for reflect in reflection_list:
    types.add(reflect[2])
    if reflect[2] == '47':
        cnt_47 += 1

In [10]:
# Total nums of different edges
len(types)

86

In [11]:
# Nums of edges with weight "47"
cnt_47

6864

## Q4 统计节点度分布,画出度分布图。

## Q5 找出给定节点i和j之间的所有最短路径，并输出。

#### （8,309）、（67,850）、（990,1256）

In [12]:
# Dijkstra algorithm
M = 1e20

def generate_matrix(node_cnt, array=[]) -> list:
    # Initialize node_cnt * node_cnt Matrix
    matrix = [[M for k in range(int(node_cnt))] for i in range(int(node_cnt))]
    for i in array:
        matrix[i[0]][i[1]] = 1
    return matrix
 
def min_index(arr=[], indexs=[]) -> int:
    # Calculate the node ordinal of the shortest path in a given node
    min_value = arr[0]
    index = indexs[0]
    for i in indexs:
        if arr[i] < min_value:
            min_value = arr[i]
            index = i
    return index
 
def dijkstra(matrix, origin) -> tuple:
    # Use Dijkstra algorithm to find the shortest path in a directed graph
    assert 0 <= origin < len(matrix) 
    s = [origin]
    dist = matrix[origin]
    vs = [i for i in range(len(dist))]
    vs.remove(origin)
    
    p = [-1 if i == M else origin for i in dist]
 
    while len(vs) > 0:
        
        node = min_index(arr=dist, indexs=vs)
        s.append(node)
        vs.remove(node)
        for i in vs:
            if matrix[node][i] != M:
                if dist[node] + matrix[node][i] < dist[i]:
                    dist[i] = dist[node] + matrix[node][i]
                    p[i] = node
    
    dist[origin] = 0
    return dist, p
 
def route(p=[]) -> list:
    i = 0
    result = []
    while i < len(p):
        m = len(p) - 1 - i
        path = [m]
        while p[m] != -1:
            path.append(p[m])
            m = p[m]
        path.reverse()
        result.append((len(p) - 1 - i, path))
        i += 1
    return result
 
if __name__ == '__main__':
    node_cnt = len(vector_set)
    array = []
    for i in reflection_list:
        array.append((int(i[0]), int(i[1])))
    matrix = generate_matrix(node_cnt=node_cnt, array=array)
    start = int(input(u'请输入起始位置（0 ~ %d）：' % (int(node_cnt - 1))))
    dist, p = dijkstra(matrix, start)
    target = int(input(u'请输入终点位置（0 ~ %d）：' % (int(node_cnt - 1))))
    for item in route(p):
        if item[0] == target:
            if dist[item[0]] != M:
                print("%d 点到 %d 点的最短距离为 %d，走过的路径为：%s" % (start, item[0], dist[item[0]], item[1]))
            else:
                print("%d 点到 %d 点不可达" % (start, item[0]))

请输入起始位置（0 ~ 1602）：8
请输入终点位置（0 ~ 1602）：309
8 点到 309 点不可达


## Q6 求所有节点的聚集系数以及平均聚集系数

In [13]:
# Ei = sum（edge(node2node)）bewteen every node connected with current node i
# Ci = (2Ei)/(ki(ki-1))

In [17]:
cites = {}
eis = {}

for item in vector_set:
    edges_node = []
    ei = 0
    for reflect in reflection_list:
        if item == reflect[0]:
            edges_node.append(reflect[1])
        elif item == reflect[1]:
            edges_node.append(reflect[0])
    cites[item] = edges_node
    
    for node in edges_node:
        for reflect in reflection_list:
            if node == reflect[0] and reflect[1] in edges_node:
                ei += 1
    eis[item] = ei

eis

{'1423': 39,
 '199': 0,
 '118': 1029,
 '67': 299,
 '542': 956,
 '198': 113,
 '263': 504,
 '1293': 13,
 '182': 201,
 '1530': 0,
 '1488': 0,
 '1560': 0,
 '447': 11,
 '244': 0,
 '725': 69,
 '310': 63,
 '351': 7,
 '627': 382,
 '21': 208,
 '830': 19,
 '1089': 540,
 '460': 152,
 '285': 673,
 '870': 1070,
 '112': 11,
 '250': 0,
 '1504': 170,
 '1532': 28,
 '561': 428,
 '128': 636,
 '1443': 43,
 '1004': 163,
 '1432': 86,
 '1259': 0,
 '1139': 22,
 '944': 562,
 '1349': 431,
 '57': 107,
 '305': 1019,
 '902': 7,
 '1240': 0,
 '695': 53,
 '467': 132,
 '145': 108,
 '1151': 19,
 '1511': 5,
 '254': 99,
 '1576': 117,
 '295': 350,
 '897': 240,
 '222': 260,
 '33': 989,
 '1128': 26,
 '653': 517,
 '1591': 63,
 '304': 227,
 '877': 95,
 '492': 640,
 '101': 217,
 '1373': 5,
 '1065': 135,
 '1167': 170,
 '822': 140,
 '60': 21,
 '721': 0,
 '1424': 72,
 '1012': 332,
 '354': 66,
 '934': 291,
 '342': 8,
 '116': 787,
 '221': 339,
 '987': 2,
 '1371': 15,
 '1122': 153,
 '1437': 44,
 '588': 59,
 '982': 0,
 '952': 468,
 '

In [21]:
# Calculate clustering coefficient with eis base on formula given above
clus_coefficient = {}
for item in eis:
    if len(cites[item])>1:
        clus_coefficient[item] = (2*eis[item])/(len(cites[item])*(len(cites[item])-1))
    else:
        clus_coefficient[item] = 0.0
clus_coefficient

{'1423': 0.16883116883116883,
 '199': 0.0,
 '118': 0.06316758747697974,
 '67': 0.10491228070175439,
 '542': 0.08441501103752759,
 '198': 0.16966966966966968,
 '263': 0.1411764705882353,
 '1293': 0.12380952380952381,
 '182': 0.08567774936061381,
 '1530': 0.0,
 '1488': 0.0,
 '1560': 0.0,
 '447': 0.09166666666666666,
 '244': 0.0,
 '725': 0.18253968253968253,
 '310': 0.1794871794871795,
 '351': 0.10606060606060606,
 '627': 0.2087431693989071,
 '21': 0.10999471179270227,
 '830': 0.1,
 '1089': 0.1348314606741573,
 '460': 0.09523809523809523,
 '285': 0.0802623732856291,
 '870': 0.062191223481546064,
 '112': 0.0718954248366013,
 '250': 0.0,
 '1504': 0.102843315184513,
 '1532': 0.10144927536231885,
 '561': 0.08646464646464647,
 '128': 0.12347117064647642,
 '1443': 0.1225071225071225,
 '1004': 0.08619777895293496,
 '1432': 0.07312925170068027,
 '1259': 0.0,
 '1139': 0.16176470588235295,
 '944': 0.06914370078740158,
 '1349': 0.07744833782569632,
 '57': 0.1371794871794872,
 '305': 0.08763329893360

In [ ]:
## Q7 